## Azure AI Foundry Agent Service: Logic App tools

This notebook demonstrates the setup and use of Azure Logic Apps as a tool for Azure AI Foundry's Agent Service, using function calling.

### Environment Setup

In [1]:
# Import Required Libraries
import os
import json
import requests
from typing import Dict, Any, Callable, Set
from datetime import datetime

from azure.identity import DefaultAzureCredential
from azure.mgmt.logic import LogicManagementClient
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ListSortOrder, ToolSet, FunctionTool

In [2]:
# Set environment variables
PROJECT_ENDPOINT = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("AZURE_FOUNDRY_GPT_MODEL")
SUBSCRIPTION_ID = os.environ.get("AZURE_SUBSCRIPTION_ID")
RESOURCE_GROUP = os.environ.get("RESOURCE_GROUP_NAME", "App_LogicApp")

LOGIC_APP_NAME = "Laziz_DemoConsLA"  # Replace with your Logic App name
TRIGGER_NAME = "HTTP-Trigger"  # Your trigger name (from your Logic App definition)

### Helper Class and Functions

In [3]:
# Azure Logic App Tool Class
class AzureLogicAppTool:
    """
    A service that manages Logic Apps by retrieving their callback URLs and invoking them.
    """

    def __init__(self, subscription_id: str, resource_group: str, credential=None):
        if credential is None:
            credential = DefaultAzureCredential()
        self.subscription_id = subscription_id
        self.resource_group = resource_group
        self.logic_client = LogicManagementClient(credential, subscription_id)
        self.callback_urls: Dict[str, str] = {}

    def register_logic_app(self, logic_app_name: str, trigger_name: str) -> None:
        """
        Retrieves and stores a callback URL for a specific Logic App + trigger.
        """
        try:
            callback = self.logic_client.workflow_triggers.list_callback_url(
                resource_group_name=self.resource_group,
                workflow_name=logic_app_name,
                trigger_name=trigger_name,
            )

            if callback.value is None:
                raise ValueError(f"No callback URL returned for Logic App '{logic_app_name}'.")

            self.callback_urls[logic_app_name] = callback.value
            print(f"Successfully registered Logic App '{logic_app_name}' with trigger '{trigger_name}'")
            
        except Exception as e:
            print(f"Error registering Logic App: {e}")
            raise

    def invoke_logic_app(self, logic_app_name: str, payload: Dict[str, Any]) -> Dict[str, Any]:
        """
        Invokes the registered Logic App with the given JSON payload.
        """
        if logic_app_name not in self.callback_urls:
            raise ValueError(f"Logic App '{logic_app_name}' has not been registered.")

        url = self.callback_urls[logic_app_name]
        
        try:
            response = requests.post(url=url, json=payload, timeout=30)
            
            if response.ok:
                try:
                    # Try to parse JSON response
                    weather_data = response.json()
                    return {"result": "success", "weather_data": weather_data}
                except json.JSONDecodeError:
                    # If not JSON, return the text
                    return {"result": "success", "data": response.text}
            else:
                return {
                    "result": "error", 
                    "error": f"HTTP {response.status_code}: {response.text}"
                }
                
        except requests.exceptions.RequestException as e:
            return {"result": "error", "error": f"Request failed: {str(e)}"}

In [4]:
# Helper Function - Weather Forecast
def create_weather_forecast_function(logic_app_tool: AzureLogicAppTool, logic_app_name: str) -> Callable[[str], str]:
    """
    Creates a weather forecast function that can be used by the AI agent.
    """
    
    def get_weather_forecast(location: str) -> str:
        """
        Gets weather forecast for a specific location using the Logic App.
        
        :param location: The location to get weather forecast for (e.g., "London", "New York", "Tokyo")
        :return: A JSON string containing the weather forecast information
        """
        payload = {"Location": location}
        
        try:
            result = logic_app_tool.invoke_logic_app(logic_app_name, payload)
            
            if result.get("result") == "success":
                weather_info = result.get("weather_data", result.get("data", "Weather data received"))
                return json.dumps({
                    "status": "success",
                    "location": location,
                    "forecast": weather_info
                })
            else:
                return json.dumps({
                    "status": "error",
                    "location": location,
                    "error": result.get("error", "Unknown error occurred")
                })
                
        except Exception as e:
            return json.dumps({
                "status": "error",
                "location": location,
                "error": str(e)
            })
    
    return get_weather_forecast

In [5]:
# Helper Function - Timer
def get_current_datetime() -> str:
    """
    Returns the current date and time in a readable format.
    """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [6]:
# Helper Function - AI Agent Interaction
def ask_weather_agent(question: str) -> None:
    """
    Ask the weather agent a question and display the response.
    """
    print(f"\nUser: {question}")
    print("=" * 50)
    
    try:
        # Create a message in the thread (using the global agents_client)
        message = agents_client.messages.create(
            thread_id = thread.id,
            role = "user",
            content = question,
        )
        
        # Create and process the agent run
        run = agents_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
        
        if run.status == "completed":
            # Get the latest messages
            messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.DESCENDING)
            
            # Find the agent's response
            for msg in messages:
                if msg.role == "assistant" and msg.text_messages:
                    latest_response = msg.text_messages[-1]
                    print(f"Agent: {latest_response.text.value}")
                    break
        else:
            print(f"Run failed with status: {run.status}")
            if run.last_error:
                print(f"Error: {run.last_error}")
    
    except Exception as e:
        print(f"Error during conversation: {e}")
        
    print("=" * 50)

### Components Activation

In [7]:
# Initialise the Logic App tool
print("Initialising Logic App tool...")
logic_app_tool = AzureLogicAppTool(SUBSCRIPTION_ID, RESOURCE_GROUP)

Initialising Logic App tool...


In [8]:
# Register the weather Logic App
print("Registering Logic App...")
logic_app_tool.register_logic_app(LOGIC_APP_NAME, TRIGGER_NAME)

Registering Logic App...
Successfully registered Logic App 'Laziz_DemoConsLA' with trigger 'HTTP-Trigger'


In [9]:
# Create the weather forecast function
print("Creating weather forecast function...")
weather_forecast_func = create_weather_forecast_function(logic_app_tool, LOGIC_APP_NAME)

print("Setup complete!")

Creating weather forecast function...
Setup complete!


### Direct Tool Testing

In [10]:
# Test the weather function directly
print("Testing weather forecast function...")
test_location = "London"
result = weather_forecast_func(test_location)

print(f"Test result for {test_location}:")
print(json.dumps(json.loads(result), indent=2))

Testing weather forecast function...
Test result for London:
{
  "status": "success",
  "location": "London",
  "forecast": {
    "responses": {
      "daily": {
        "day": {
          "cap": "Rain showers",
          "pvdrCap": "Rain showers",
          "pvdrWindDir": "161",
          "pvdrWindSpd": "20",
          "icon": 23,
          "symbol": "d2200",
          "pvdrIcon": "23",
          "urlIcon": "http://img-s-msn-com.akamaized.net/tenant/amp/entityid/AAehOqC.img",
          "precip": 100.0,
          "wx": "SHRA",
          "sky": "BKN",
          "windDir": 161,
          "windSpd": 20.0,
          "summary": "Watch for scattered rain showers.  The high will be 19\u00b0.",
          "summaries": [
            "Watch for scattered rain showers.",
            " The high will be 19\u00b0."
          ]
        },
        "night": {
          "cap": "Light rain showers",
          "pvdrCap": "Light rain showers",
          "pvdrWindDir": "193",
          "pvdrWindSpd": "17",
 

### AI Agent Setup

In [11]:
# Create the AI Project client
print("Creating AI Project client...")
project_client = AIProjectClient(
    endpoint = PROJECT_ENDPOINT,
    credential = DefaultAzureCredential(),
)

Creating AI Project client...


In [12]:
# Prepare function tools for the agent
functions_to_use: Set = {
    get_current_datetime,
    weather_forecast_func,
}

print("Setting up agent tools...")
agents_client = project_client.agents

# Create function tools
functions = FunctionTool(functions=functions_to_use)
toolset = ToolSet()
toolset.add(functions)

# Enable automatic function calls
agents_client.enable_auto_function_calls(toolset)

# Create the weather agent
print("Creating weather agent...")
agent = agents_client.create_agent(
    model = MODEL_DEPLOYMENT,
    name = "WeatherForecastAgent",
    instructions = """You are a helpful weather assistant agent. You can:
    1. Get current date and time
    2. Provide weather forecasts for any location using the Logic App
        
    When users ask for weather information, use the get_weather_forecast function with the location they specify.
    Present the weather information in a friendly and readable format.
    Always include the current date and time when providing forecasts.""",
    toolset = toolset,
)
print(f"Created weather agent with ID: {agent.id}")

# Create a conversation thread
print("Creating conversation thread...")
thread = agents_client.threads.create()
print(f"Created thread with ID: {thread.id}")

print("Agent is ready! You can now interact with it.")

Setting up agent tools...
Creating weather agent...
Created weather agent with ID: asst_P24uYTY3zZhH2VFnP6on62lt
Creating conversation thread...
Created thread with ID: thread_au3fmRys3MadIZTnCppkDHmf
Agent is ready! You can now interact with it.


### AI Agent Testing

In [13]:
# Example 1: Simple weather query
ask_weather_agent("What's the weather like in Paris today?")


User: What's the weather like in Paris today?
Agent: As of today, September 10, 2025, in Paris:

- Expect scattered rain showers throughout the day.
- The high temperature will reach around 21°C.
- Wind will be coming from the south-southwest at about 15 km/h.
- During the night, rain showers will continue with a low around 13°C and winds at 13 km/h from the south.

So, it's a rainy day with moderate temperatures. Make sure to carry an umbrella if you are heading out!

If you need more detailed information or forecast for another day, just let me know!


In [14]:
# Example 2: Multiple locations
ask_weather_agent("Can you tell me the weather forecast for Tokyo and New York?")


User: Can you tell me the weather forecast for Tokyo and New York?
Agent: Here is the weather forecast for today, September 10, 2025:

Tokyo:
- Scattered rain showers during the day with a high of 30°C, quite humid.
- Heavy rain expected at night with a low of 23°C.
- Winds from the north-northeast at 18 km/h during the day and from the northeast at 16 km/h at night.

New York:
- Partly sunny skies during the day with a high of 23°C.
- Mostly clear skies at night with a low of 16°C.
- Winds coming from the northeast at 10 km/h during the day and from the northwest at 4 km/h at night.

Let me know if you'd like more details or forecasts for other locations!


In [15]:
# Example 3: Weather with time context
ask_weather_agent("What's the current time and weather forecast for Sydney?")


User: What's the current time and weather forecast for Sydney?
Agent: The current date and time is September 10, 2025.

In Sydney today:
- Heavy rain is expected during the daytime hours.
- The high temperature will be around 16°C on this breezy day with winds coming from the southwest at 32 km/h.
- At night, the skies will be partly cloudy with a low of 10°C and winds at 14 km/h from the west-southwest.

If you need more information or forecasts for another location, feel free to ask!


### Housekeeping

In [16]:
# Delete the thread and agent when finished
try:
    # Delete the thread
    agents_client.threads.delete(thread.id)
    print("Deleted thread successfully")

    # Use the existing agents_client (no need for 'with' block)
    agents_client.delete_agent(agent.id)
    print("Deleted weather agent successfully")
    
    # Close the project client
    project_client.close()
    print("Closed project client connection")
    
except Exception as e:
    print(f"Error during cleanup: {e}")

print("Cleanup complete!")

Deleted thread successfully
Deleted weather agent successfully
Closed project client connection
Cleanup complete!
